<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to DSCI 101 #
### Introduction to Data Science ###

### Code Demo for Week 1 ###

In [ ]:
# standard import Python modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.rcParams['patch.force_edgecolor'] = True

%matplotlib inline

# This demo is a toy example of what you will do in this class: 
## Question $\rightarrow$ Data $\rightarrow$ Answer $\rightarrow$ More questions $\rightarrow$ ...


## Your ultimate goal is to walk through your own Data Science Pipeline by completing a real data project.


<br/><br/><br/><br/>

## Question: Can we predict gender by first name?

How can we answer this question?

<br/><br/><br/><br/>


## Ideas:

1. What do we mean by gender? 
1. Can we use the name to predict gender?
1. How would we build a model to predict gender given the first name?
1. Where can we get data for such a model?



<br/><br/><br/><br/><br/>


# US Social Security Data

Public dataset containing baby names and their **biological sex**.

## Understanding the Setting

In this course you will have to find interesting dataset to work with, and learn about different data sources on your own. **It is super important to read data documentation!** 

<img src="baby_names_desc.png" width="600px" />

Reading from [SSN Office description](https://www.ssa.gov/oact/babynames/background.html): 

> All names are from Social Security card applications for births that occurred in the United States after 1879. Note  that many people born before 1937 never applied for a Social Security card, so their names are not included in our data. For others who did apply, our records may not show the place of birth, and again their names are not included in our data.

> The data we are using below are from a 100% sample of the records on Social Security card applications as of end of year when the database was last updated.

## Get data programatically

This is automated pipeline to download data and consolidate files into one big data table. SSN data is updated every year, and this ensures your data always include the most recent year.

In [ ]:
# don't worry about the code for now
# what it is doing is to download the zip file of data from www.ssa.gov
# then read in all data files and put them together as a big data table
import urllib.request
import os.path

# Download data from the web directly: this database is updated annually
# data source location: https://www.ssa.gov/oact/babynames/limits.html
data_url = 'https://www.ssa.gov/oact/babynames/names.zip'
local_filename = 'names.zip'
# if the data already exists then no need to download again
if not os.path.exists(local_filename): 
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

        
# Load data without unzipping the file
import zipfile
babynames = [] 
with zipfile.ZipFile(local_filename, 'r') as zf:
    data_files = [f for f in zf.filelist if f.filename[-3:] == 'txt']
    ### a function to return year in filename
    def extract_year_from_filename(fn):
        # return the year in filename
        return int(fn[3:7])
    ### end of the function
    # loop over all the files in zip
    for f in data_files:
        # extract year from filename
        year = extract_year_from_filename(f.filename)
        with zf.open(f) as fp:
            # create a dataframe: order of columns in data files
            df = pd.read_csv(fp, names=['Name', 'Sex', 'Count'])
            # add a column for year
            df['Year'] = year
            babynames.append(df)
babynames = pd.concat(babynames)

# the big table is called babynames
# let's take a look at what information it contains
babynames

## A little bit of data cleaning
This data is already super clean, thus does not require lots of wrangline or preprocessing. 

In [ ]:
# this converts all names to lower case
babynames['Name'] = babynames['Name'].str.lower()
babynames

In [ ]:
# this returns the number of rows and columns of a data frame
babynames.shape

Make sure you understand the rows and columns in this `babynames` dataframe:
- What does each row stand for?
- Are the column names self-explanatory?

In [ ]:
# this returns all records with name you specify for ...
# fill in your name or whatever name you would like to look up
# make sure you put quotes around name like "sue"
babynames[babynames['Name']==...]

In [ ]:
# this returns all records with name "..." for the birth year XXXX
babynames[(babynames['Name']==...)&(babynames['Year']==...)]

In [ ]:
# fill in Names, Sex to be M or F
# make a plot
# what is this plot about?
Name = ...
Sex = ...
df = babynames.loc[(babynames['Name']==Name)&(babynames['Sex']==Sex),['Count', 'Year']]
df.plot(x='Year', y='Count', title='Number of babies with name '+Name+' and sex '+Sex);

## Understand our data before using: 

### How many babies for each year?

We will learn to create pivot tables in this class.

In [ ]:
# a pivot table is a great way to summarize data
pivot_year_name_count = babynames.pivot_table(
        index='Year', # the row index
        columns='Sex', # the column values
        values='Count', # the field(s) to processed in each group
        aggfunc='sum' # the aggregate function to apply to values
    )

# what does this table show?
pivot_year_name_count

In [ ]:
# make a plot again!
pivot_year_name_count.plot()
plt.ylabel('number of babies')
plt.title('Number of Babies by Year');

### How many unique names for each year?

In [ ]:
# another pivot table
# notice the difference?
# the aggregate function here is "count" instead of "sum"
pivot_year_name_unique = babynames.pivot_table(
        index='Year', # the row index
        columns='Sex', # the column values
        values='Name', # the field(s) to processed in each group
        aggfunc='count' # the aggregate function to apply to values
    )
pivot_year_name_unique

In [ ]:
pivot_year_name_unique.plot()
plt.ylabel('number of names')
plt.title('Number of Unique Names by Year');

### Compute the proportion of male and female babies for each name

In [ ]:
# more pivot table!
sex_counts = babynames.pivot_table(
        index='Name', 
        columns='Sex', 
        values='Count',
        aggfunc='sum', fill_value=0, margins=True)
sex_counts

Compute proportion of female babies given each name.

In [ ]:
# return a list of proportion of female for each name
prop_female = sex_counts['F'] / sex_counts['All'] 
prop_female

In [ ]:
# look at first # rows
prop_female.head(...)

In [ ]:
# look at last # rows
prop_female.tail(...)

# Testing a few Names

In [ ]:
# try out your name!
prop_female[...]

In [ ]:
# try a few names here
# most gender neutral name winner???
prop_female[...]

In [ ]:
gn_name = ...
df = babynames[(babynames['Name']==gn_name)]
pivot = df.pivot_table(
        index='Year', # the row index
        columns='Sex', # the column values
        values='Count', # the field(s) to processed in each group
        aggfunc='sum', # the aggregate function to apply to values
        fill_value=0
    )
pivot.plot(title='Most gender neutral name: '+gn_name);

## Build Simple Classifier (Model)

In [ ]:
# a function takes a name, and return "F" or "M"
# based on historical female proportion of female of that name
def sex_from_name(name):
    lower_name = name.lower()
    if lower_name in prop_female.index:
        return 'F' if prop_female[lower_name] > 0.5 else 'M'
    else:
        return 'Unknown'

In [ ]:
# test with a name using this function
sex_from_name(...)

## What fraction of people in this class have names in the SSN Database?

### Let's first load and clean the people names I downloaded from Canvas 

In [ ]:
# DSCI101_names.csv is a file I downloaded from registra
names = pd.read_csv('DSCI101_names.csv')
# convert names to lower case
names['First Name'] = names['First Name'].str.lower()
names['Last Name'] = names['Last Name'].str.lower()
# how many total names
print('Nubmer of people: ', len(names))
# first few rows
names.head()

In [ ]:
student_names = pd.Index(names['First Name']).intersection(prop_female.index)
print('Fraction of names in the babynames data:' , round(len(student_names) / len(names), 2))

## Which name/names are not in the Database?

Why might these names not appear?  

In [ ]:
missing_names = pd.Index(names['First Name']).difference(prop_female.index)
missing_names

# Now we can use the model to predict!
## input: your name
## model: sex_from_name
## output: model predicted sex for you

In [ ]:
names['Predicted Sex'] = names['First Name'].apply(sex_from_name)
names

## How did our model do??? How do we know if the model is good or not?

# Estimate the fraction of female and male students

In [ ]:
pred_prob_female = np.mean(names[names['Predicted Sex'] != 'Unknown']['Predicted Sex']=='F')
pred_prob_female

# Using Simulation to Estimate Uncertainty
## What do I mean by uncertainty?

Remember how the classifier `sex_from_name` work?
We calculate the female proportion of a given name in the SSD Database, and if that proportion is greater than 0.5, we predict this person as F, otherwise M. (For names not found in the database, predict "unknown".)

Suppose we have two names, name1 has female proportion 0.50000001, and name2 has female proportion 0.49999999. What should we predict? This means when you predict a gender neutral name, you could be wrong! That is uncertainty.

## Restricting our attention to Students in the Class

In [ ]:
DSCI_prob_female = prop_female.loc[prop_female.index.intersection(names['First Name'])]
DSCI_prob_female

# Running the Simulation

## The idea:
Once I calculate a proportion of female for a particular name, I will not assign deterministic sex based on hard threshold 0.5, insted, I will randomly assign gender F with probability equals the female proportion I calculated from the data.

Thanks to computer random number generator, this is possible (Think about lottery, the idea is pretty much the same). If the female proportion for a particular name is large, it is more likely to be predicted as female by my model. Does that make sense?

For example:

In [ ]:
# randomly select from a with replacement size= objects with probability p
np.random.choice(a=['F', 'M'], size=1, p=[0.8, 0.2])

In [ ]:
# now use a particular female proportion as probability
# pick a name that the pro_female is not 0 or 1
fp = DSCI_prob_female[...]
mp = 1 - fp
np.random.choice(a=['F', 'M'], size=1, p=[fp, mp])

In [ ]:
# do this for all students in DSCI 101
# an empty list to same all model predicted sex
predict_sex = []

# a for loop to loop over all the names in DSCI 101

# find female proportion for each name
for fp in DSCI_prob_female:
    mp = 1 - fp
    # randomly predict based on fp
    sex = np.random.choice(a=['F', 'M'], size=1, p=[fp, mp])
    # save the result
    predict_sex = np.append(predict_sex, sex)

# print out results
predict_sex

In [ ]:
# calculat predicted female proportion in the class
np.mean(predict_sex == 'F')

In [ ]:
# function that performs one simulation
def simulate_female_prop(class_fp):
    predict_sex = []

    for fp in class_fp:
        mp = 1 - fp
        sex = np.random.choice(a=['F', 'M'], size=1, p=[fp, mp])
        predict_sex = np.append(predict_sex, sex)
        
    return np.mean(predict_sex == 'F')

In [ ]:
# run the function once using DSCI 101 female prob
simulate_female_prop(class_fp = DSCI_prob_female)

In [ ]:
# performs many simulations
rep = 10000
# an empty list to save all the results
simulations = []

# a for loop to repeat the same simulation rep times
for n in range(rep):
    one_simu = simulate_female_prop(class_fp = DSCI_prob_female)
    simulations = np.append(simulations, one_simu)

In [ ]:
# what do you expect to see in this results?
simulations

In [ ]:
# a histogram of estimated female proportion in DSCI 101
plt.hist(simulations)
plt.xlabel('% of female')
plt.ylabel('number of simulation')
plt.title('estimated % of female in DSCI 101 by first name');

## Is this model good enough?
## How can you improve it without more data?
## What other quetions can you ask about the data?
## What other data out there may help anwser your questions or improve your model?